In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [2]:
spx_path = "data/SPX_database_2010.xlsx"
df_tri = pd.read_excel(spx_path, sheet_name="total_ret").set_index("Dates")

df_sofr = pd.read_excel("./data/sofr.xlsx")
df_sofr.rename(columns={"Data": "Dates"}, inplace=True)
df_sofr["Dates"] = pd.to_datetime(df_sofr["Dates"]) 
df_sofr = df_sofr.set_index("Dates").sort_index()
df_sofr["Taxa_diaria"] = ((1 + df_sofr["Price"]) ** (1/252)) -1
df_sofr

,Price,Taxa_diaria
Dates,,
2014-01-02,0.0005,0.000002
2014-01-03,0.0005,0.000002
2014-01-06,0.0005,0.000002
2014-01-07,0.0004,0.000002
2014-01-08,0.0004,0.000002
...,...,...
2025-07-28,0.0436,0.000169
2025-07-29,0.0436,0.000169
2025-07-30,0.0432,0.000168


In [3]:
# df_tri = df_tri[(df_tri.index.year >= 2000)]
df_sofr = df_sofr[(df_sofr.index.year >= 2000) ]
prices = df_tri.copy()
returns = prices.pct_change()

/var/folders/pm/ps47mfm943x_nqjshcjsk5ch0000gn/T/ipykernel_47562/850101438.py:4: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change()


In [4]:
def _choose_30(tri_df, random_state=42):
    eligible = sorted(list(map(str, tri_df.columns)))
    rng = np.random.default_rng(random_state)
    chosen = rng.choice(eligible, size=min(30, len(eligible)), replace=False)
    return list(map(str, chosen))

tickers_30 = _choose_30(prices)
tickers_30[:5]

['MSI UN Equity.1',
 'K UN Equity.1',
 'GRMN UN Equity',
 'SW UN Equity.1',
 'AVGO UW Equity.1']

In [ ]:
prices = prices[tickers_30]
returns = returns[tickers_30]
prices = df_tri.copy()
prices.index = pd.to_datetime(prices.index)
prices = prices.select_dtypes(include=[np.number]).dropna(how="all", axis=1)